In [1]:
import urllib
from bs4 import BeautifulSoup
from pandas.io.parsers import TextParser
from lxml.html import parse 
from urllib.request import urlopen
import csv
import pandas as pd

In [2]:
url1 = ""
url_pre = "https://www.sec.gov/Archives/edgar/data/"
def geturl(cik, accno):
    cik_striped = cik.lstrip("0")
    accno_striped = accno.replace("-","")
    url1 = url_pre + cik_striped + "/" + accno_striped + "/" + accno + "-index.html"
    return url1

geturl("000005114sss3", "0000051143-13-000007")


'https://www.sec.gov/Archives/edgar/data/5114sss3/000005114313000007/0000051143-13-000007-index.html'

In [21]:
# retrieve the web page
page = urllib.request.urlopen(geturl("0000051143", "0000051143-13-000007"))
# parse the page and save it in soup format 
soup = BeautifulSoup(page,"lxml")
#print(soup.prettify)
url2 = ""

form = soup.find(id='formName').get_text()
formname = form[6:10]
print(formname)

#formtype = soup.find(text=formname)
formtype = soup.findAll('td', text = formname)[0]

formprevious = soup.find_previous_sibling('td', formtype)

print(formtype)

#all_links = soup.find_all('a')
#for link in all_links:
#    if "10q.htm" in link.get("href"):
#        url2 = "https://www.sec.gov/" + link.get("href")
        
#print(url2)

10-Q
<td scope="row">10-Q</td>


In [4]:
# retrieve the web page
print(url2)
page_10q = urllib.request.urlopen(url2)
soup = BeautifulSoup(page_10q,"lxml")
#print(soup.prettify)
#parsed = parse(urlopen(url2))

#doc = parsed.getroot()
#all_tables=doc.findall('.//table')
#all_tables = all_tables[4]

https://www.sec.gov//Archives/edgar/data/51143/000005114313000007/ibm13q3_10q.htm


In [6]:
all_tables= soup.select('div["bclpageborder"] table')
#all_tables = soup.findAll('table')
for tab in all_tables:
    records = []
    for tr in tab.find_all('tr'):
        rowString=[]
        for td in tr.findAll('td'):
            p = td.find_all('p')
            if len(p)>0:
                for ps in p:
                    ps_text = ps.get_text().replace("\n"," ") 
                    ps_text = ps_text.replace("\xa0","")                 
                    rowString.append(ps_text)
            else:
                td_text=td.get_text().replace("\n"," ")
                td_text = td_text.replace("\xa0","")
                rowString.append(td_text)
        records.append(rowString)        
    with open(str(all_tables.index(tab)) + 'tables.csv', 'w') as f:
        writer = csv.writer(f)
        writer.writerows(records)
    